In [ ]:
import pdfplumber
import pandas as pd
import numpy as np

# Đọc file PDF
file_path = "../data/data.pdf"

data = []
with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            data.extend(table)

# Chuyển đổi dữ liệu thành DataFrame
columns = ["STT", "MSSV", "Họ và tên lót", "Tên", "Khoa", "Ngành",
           "ĐRL HK1", "ĐRL HK2", "ĐTB RL NH", "Xếp loại RL",
           "TBHT HK1", "TBHT HK2", "TBHT NH", "Xếp loại HT", "Đề nghị Danh hiệu"]
df = pd.DataFrame(data[1:], columns=columns)

# Kiểm tra dữ liệu
print(df.head())

# Chuyển các giá trị thập phân từ dấu phẩy sang dấu chấm
def convert_decimal(value):
    try:
        return float(value.replace(',', '.'))
    except:
        return np.nan

numeric_columns = ["ĐRL HK1", "ĐRL HK2", "ĐTB RL NH", "TBHT HK1", "TBHT HK2", "TBHT NH"]
for col in numeric_columns:
    df[col] = df[col].apply(convert_decimal)

# Loại bỏ các hàng không hợp lệ
df = df.dropna(subset=["ĐTB RL NH", "TBHT NH", "Đề nghị Danh hiệu"])

# Đổi tên các cột để dễ thao tác
df = df.rename(columns={
    "ĐTB RL NH": "average_behavior_score",
    "TBHT NH": "average_academic_score",
    "Đề nghị Danh hiệu": "label"
})

# Giữ lại các cột quan trọng
df = df[["average_behavior_score", "average_academic_score", "label"]]

# Kiểm tra dữ liệu sau khi xử lý
print(df.head())


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Mã hóa nhãn (label)
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Tách dữ liệu thành tập train và test
X = df[["average_behavior_score", "average_academic_score"]]
y = df["label_encoded"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Lưu thông tin để sử dụng sau
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping:", label_mapping)


              precision    recall  f1-score   support

           0       0.57      0.80      0.67         5
           2       0.86      0.67      0.75         9

    accuracy                           0.71        14
   macro avg       0.71      0.73      0.71        14
weighted avg       0.76      0.71      0.72        14

Mapping: {'Học bổng SV nỗ lực': 0, 'Học bổng SV tài năng': 1, 'Khen thưởng SV giỏi': 2}


In [6]:
import joblib
joblib.dump(model, "trained_model.pkl")

['trained_model.pkl']

In [7]:
import streamlit as st
import numpy as np

# Load mô hình và mapping nhãn
import joblib
model = joblib.load("trained_model.pkl")
label_mapping = {'Học bổng SV nỗ lực': 0, 'Học bổng SV tài năng': 1, 'Khen thưởng SV giỏi': 2}
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Tiêu đề ứng dụng
st.title("Dự Đoán Danh Hiệu Sinh Viên")

# Nhập liệu từ người dùng
behavior_score = st.number_input("Nhập điểm rèn luyện (ĐRL)", min_value=0.0, max_value=100.0, step=0.1)
academic_score = st.number_input("Nhập GPA (TBHT)", min_value=0.0, max_value=4.0, step=0.1)

# Dự đoán khi người dùng nhấn nút
if st.button("Dự Đoán"):
    # Chuẩn hóa dữ liệu
    input_data = np.array([[behavior_score, academic_score]])
    prediction = model.predict(input_data)
    predicted_label = reverse_label_mapping[prediction[0]]
    
    # Hiển thị kết quả
    st.success(f"Loại danh hiệu dự đoán: {predicted_label}")


2024-12-07 22:04:53.748 
  command:

    streamlit run C:\Users\ihatesea\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [8]:
streamlit run app.py


SyntaxError: invalid syntax (507122745.py, line 1)